# Project 2: LangChain RAG Project
## Developed By Rohail Tabassum


### Installing Packages of Langchain Pinecone and Google Gemini LLM

In [107]:
%pip install -qU langchain-pinecone langchain-google-genai

### Created account on Pinecone Platform, Got Pinecone API Key, Created an Pinecone object

In [108]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
import os

os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

pc = Pinecone()

### Created an Index in Pinecone

In [109]:

index_name = "first-rag-project"  # change if desired

# pc.create_index(
#     name=index_name,
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )

index = pc.Index(index_name)

### Setting up Google API KEY

In [110]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

### Setting up Google Embedding Model

In [111]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Example of how the embedding model converts text into vector


In [112]:
vector = embeddings.embed_query("hello, world!")
print(vector[:5])
print(len(vector))

[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966]
768


### Setting up Pinecone Vector Store


In [113]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Creating Dummy Documents and assigning them a unique ID

In [114]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


uuids = [str(uuid4()) for _ in range(len(documents))]


### Adding documents to Vector Store

In [115]:
vector_store.add_documents(documents=documents, ids=uuids)

['b959b4ad-1872-4578-8300-cf002e3eced9',
 '110817f2-dcb4-4840-8f00-4d23ad94587c',
 '9f2aae44-0071-4c62-974a-f5539b74acec',
 '32a56e5b-11b9-47e0-ad51-0564204802af',
 '63170993-b7da-4160-a6c1-2d79443d6cfa',
 '92f0ccc7-4e67-421a-926d-6d5cb9df5f86',
 '3a517ef6-b743-467c-b084-a542a82ab337',
 'c2834614-cff2-436b-9dbf-1cecb5503e27',
 'b960a561-1af4-496e-b908-74488fe56c91',
 'b0735b1f-753c-4b25-867a-e14d4c559576']

### Retrieving Data from Vector Store using similarity search method

In [122]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"}
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


 ### Retrieving Data from Vector Store using similarity search with score method

In [123]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.537337] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.533720] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


### Setting the Google Gemini LLM

In [118]:
from langchain_google_genai import ChatGoogleGenerativeAI

model : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp"
)

### Interacting with LLM using user query and vector results

In [124]:
def get_response_from_llm(query):

  # Vector Search
  vector_results = vector_store.similarity_search(query,k=2)

  # Pass the user query and vector results to llm

  llm_response = model.invoke(f"Answer this user query : {query}. Here are some references to assist you {vector_results}. Use this reference and your own knowledge to give optimal response to the user.")

  return llm_response


In [125]:
response = get_response_from_llm("What is LangGraph?")
print(response)

content='Okay, based on the provided references and my knowledge, here\'s an answer to "What is LangGraph?":\n\nLangGraph is a framework specifically designed for building **stateful, agentic applications**. This means it focuses on creating applications that can:\n\n*   **Maintain State:** Track information and context across multiple interactions or steps in a process. This allows the application to "remember" what it\'s done and make decisions based on that history.\n*   **Be Agentic:** Act autonomously, making decisions and taking actions based on its internal state and external inputs. This often involves using tools, APIs, or other resources to achieve a goal.\n\nWhile the provided references are brief (mainly tweets), they highlight that LangGraph is considered a leading framework for this type of application development. It\'s positioned as the best tool for building complex, multi-step processes that require intelligent decision-making.\n\n**Here\'s a breakdown of what we can 

In [126]:
print(response.content)

Okay, based on the provided references and my knowledge, here's an answer to "What is LangGraph?":

LangGraph is a framework specifically designed for building **stateful, agentic applications**. This means it focuses on creating applications that can:

*   **Maintain State:** Track information and context across multiple interactions or steps in a process. This allows the application to "remember" what it's done and make decisions based on that history.
*   **Be Agentic:** Act autonomously, making decisions and taking actions based on its internal state and external inputs. This often involves using tools, APIs, or other resources to achieve a goal.

While the provided references are brief (mainly tweets), they highlight that LangGraph is considered a leading framework for this type of application development. It's positioned as the best tool for building complex, multi-step processes that require intelligent decision-making.

**Here's a breakdown of what we can infer and know about L